In [31]:
import tensorflow as tf
import numpy as np

In [32]:
from google.colab import files
files.upload()
from datasets import MNISTDataset

Saving datasets.py to datasets (3).py


In [33]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [34]:
train_images = (train_images.astype(np.float32) / 255.).reshape((-1, 784))
test_images = (test_images.astype(np.float32) / 255.).reshape((-1, 784))

train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data = train_data.batch(128)


In [35]:
train_steps = 1000
learning_rate = 0.1

W_h=tf.Variable(tf.random.normal([784,60],mean = 0,stddev=0.5,dtype=tf.dtypes.float32))
b_h= tf.Variable(tf.random.normal( [60],mean = 0,stddev=0.5 ,dtype=tf.dtypes.float32))
W_o=tf.Variable(tf.random.normal([60,10],mean = 0,stddev=0.5 ,dtype=tf.dtypes.float32))
b_o= tf.Variable(tf.random.normal( [10], mean = 0,stddev=0.5,dtype=tf.dtypes.float32))

for step, (image_batch, label_batch) in enumerate(train_data):
  if step>train_steps:
    break

  with tf.GradientTape() as tape:
     logits_1 = tf.nn.relu(tf.matmul(image_batch, W_h) + b_h)
     logits_2 = (tf.matmul(logits_1, W_o) + b_o)
     xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logits_2, labels=label_batch))
     
     
  
     grads = tape.gradient(xent, [W_o,b_o,W_h,b_h])
     W_o.assign_sub(learning_rate * grads[0])
     b_o.assign_sub(learning_rate * grads[1])
     W_h.assign_sub(learning_rate * grads[2])
     b_h.assign_sub(learning_rate * grads[3])

  if step % 100:
      print(step)
      preds = tf.argmax(logits_2, axis=1, output_type=tf.int32)
      acc = tf.reduce_mean(tf.cast(tf.equal(preds, label_batch),
                             tf.float32))
      print("Loss: {} Accuracy: {}".format(xent, acc))

     




1
Loss: 14.289302825927734 Accuracy: 0.109375
2
Loss: 11.303542137145996 Accuracy: 0.1875
3
Loss: 9.034351348876953 Accuracy: 0.1171875
4
Loss: 8.489034652709961 Accuracy: 0.1875
5
Loss: 6.509423732757568 Accuracy: 0.2265625
6
Loss: 5.273309707641602 Accuracy: 0.2734375
7
Loss: 4.980221271514893 Accuracy: 0.2265625
8
Loss: 5.2599592208862305 Accuracy: 0.234375
9
Loss: 4.845675945281982 Accuracy: 0.3125
10
Loss: 4.852930068969727 Accuracy: 0.328125
11
Loss: 4.058111667633057 Accuracy: 0.390625
12
Loss: 3.407993793487549 Accuracy: 0.453125
13
Loss: 2.934558868408203 Accuracy: 0.5390625
14
Loss: 3.410280704498291 Accuracy: 0.5
15
Loss: 3.240185022354126 Accuracy: 0.53125
16
Loss: 2.647278070449829 Accuracy: 0.5703125
17
Loss: 2.46423602104187 Accuracy: 0.5234375
18
Loss: 3.065714120864868 Accuracy: 0.4921875
19
Loss: 2.308990955352783 Accuracy: 0.5625
20
Loss: 2.364504098892212 Accuracy: 0.640625
21
Loss: 3.0741865634918213 Accuracy: 0.5234375
22
Loss: 2.94417667388916 Accuracy: 0.4609375

In [36]:
test_preds_h = tf.matmul(test_images, W_h) + b_h
test_preds = tf.argmax(tf.matmul(test_preds_h, W_o) + b_o, axis=1,
                       output_type=tf.int32)                     
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, test_labels),
                             tf.float32))
print('test accuracy:',acc)

test accuracy: tf.Tensor(0.5262, shape=(), dtype=float32)



1. data.repeat.shuffle.batch will repeat the data,shuffle the repeated data and divide it into batches.
2. data.batch.shuffle.repeat will batch the data, shuffle the batch data and repear the shuffled batches.
3. data.shuffle.batch.repeat will shuffle the data, then divide it into batches and repeat the divided batches.
4. data.repeat.batch.shuffle will repeat the data, batch the repeated data and then shuffle these repeated batches.
5. data.shuffle.repeat.batch will first shuffle the data, then repeat the shuffled data and then divivde it into batches.
6. data.batch.repeat.shuffle will batch the data, repeat the batches of data and repeat the shuffled batches.





Fail2.py

Problem- The model is not being trained properly as the loss is constant at 2 and the accuracy is not increasing and very poor.

Fixing the problem- 

-Relu can be used instead of sigmoid activation function.


Fail1.py

Problem- nan Accuracy which is caused by vanishing gradient due to not proper weight updates of the inputs. So accuracy is very very less almost zero. It is a very dense neuralnetwork.

Fixing problem-The weight initialization should be properly done. Instead of 8 layers number of layers is decreased to 2. 




Fail3.py

Problem- The loss is constant and non decreasing also the accuracy is very very small because the input weight is initialized by 0.

Fix: The input weight can be initialized by 0.1 instead of 0.
w_input = tf.Variable(tf.random.uniform([784, n_units], -w_range, 0.1),
                      name="w0")

Fail4.py

Problem- Due to the standard deviation 2 in noise, the model is not able to detect image properly. So adding small amount of noise increases generelizatio not a lot.

Fix- Making the standrad deviation to 0.1 instead of 2 increases the accuracy. 

 img_batch += tf.random.normal(tf.shape(img_batch), stddev=0.1)


Fail5.py

Problem-In the model_forward(inputs) function in logits softmax is being used.If we use softmax in this layer all nodes (hidden variables) will be linearly dependent which may result in many problems and poor generalization.

Fix- Not using softmax in that layer. Instead use-
logits = tf.matmul(x, layers[-1][0] + layers[-1][1])